**Please only run the following cell if you're running this notebook on Google colab or another cloud hosted notebook server.**

In [0]:
!pip install qiskit

# E91 Quantum Key Distribution (QKD) Protocol

This notebook is a _demonstration_ of the E91 Protocol for QKD using Qiskit. 
E91 is a quantum key distribution scheme developed by Artur Ekert in 1991 ([paper]).

[paper]: https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.67.661


In [0]:
from qiskit import BasicAer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute
from qiskit import IBMQ


from qiskit.tools.monitor import job_monitor
%pylab inline

First we will simulate a noiseless quantum channel free from an eavesdropper. The qubit capabilites of the available backends are listed below:


| Backend | Number of qubits |
|----------|------------------|
|qasm_simulator|24|
|ibmq_qasm_simulator|32|
|ibmqx4|5|
|ibmqx2|5|
|ibmq_16_melbourne|14|

## Choosing bases

A third party creates $|B_{00}\rangle $ Bell pairs and distributes the halves to Alice and Bob. Aice and Bob each choose a basis using the following convention:

$0 \rightarrow$ Computational basis

$1  \rightarrow$ Hadamard basis

Here Eve, also choose a basis randomly using the convention above



In [0]:
num_qubits = 32


alice_basis = np.random.randint(2, size=num_qubits//2)
bob_basis = np.random.randint(2, size=num_qubits//2)
eve_basis = np.random.randint(2, size=num_qubits//2)


print(alice_basis)
print(bob_basis)
print(eve_basis)

## Creating the circuit

Based on the following results:

$CX(H \otimes I)|00\rangle = |B_{00}\rangle$


Our algorithm to construct the circuit is as follows:

1. Create Bell pairs for Alice and Bob using the above result. One half of each pair is sent to Alice and Bob respectively.

2. Eve intercepts Bob's qubits and performs a measurement on them choosing a basis randomly according to her binary string. 

3. Alice and Bob measure the qubits according to their binary strings. To measure a qubit in the Hadamard basis, they apply an $H$ gate to the corresponding qubit and then perform a mesurement on the computational basis.

In [0]:
a = QuantumRegister(num_qubits//2,'qa')
b = QuantumRegister(num_qubits//2,'qb')
ca = ClassicalRegister(num_qubits//2,'c_alice')
cb = ClassicalRegister(num_qubits//2,'c_bob')
ce = ClassicalRegister(num_qubits//2,'c_eve')

e91 = QuantumCircuit(a,b,ca,cb, ce,name='e91')

# Entangle pairs of qubits for Alice and Bob
for index in range(num_qubits//2):
    e91.h(a[index])
    e91.cx(a[index],b[index])
    
e91.barrier()    

# Eve intercepts Bob's qubits and measures them before sending them to Bob
for index, _ in enumerate(eve_basis):
    if eve_basis[index] == 1:
        e91.h(b[index])    

e91.measure(b,ce)
e91.barrier()

# Alice measures the received qubits
for index, _ in enumerate(alice_basis):
    if alice_basis[index] == 1:
        e91.h(a[index])    

e91.measure(a,ca)
e91.barrier()    

# Bob measures the received qubits
for index, _ in enumerate(bob_basis):
    if bob_basis[index] == 1:
        e91.h(b[index]) 
        
e91.measure(b,cb)
    
            
# e91.draw()


### Running it on the IBMQ machines

The following cell is only needed if you want to use the IBM Q Experience devices. Please enter your API token between the quotes. Instructions are given [here](https://github.com/Qiskit/qiskit/blob/master/docs/install.rst#access-ibm-q-devices)
Uncomment the code in the cell to use it. 

In [0]:
token = '8dad7b6e859473840931777cf93a88d380b3b1b2525c02be6b583ecfe6080e8f0c683b9b6e8810eb5dc91b2bf95ae6bbc800d7d8655c4d5d6480f89f9c1566d9'
IBMQ.enable_account(token)
IBMQ.backends()

## Performing the actual experiment

Run the code on the desired backend. Use the cell above to authenticate yourself to the IBMQ network.

In [0]:
# backend = BasicAer.get_backend('qasm_simulator')
# backend = IBMQ.get_backend('ibmq_16_melbourne')
backend = IBMQ.get_backend('ibmq_qasm_simulator')

job = execute(e91,backend,shots=8192)

job_monitor(job)

result = job.result()
counts = result.get_counts()

# Prints out all the measured binary strings by Bob (reversed)
for count in [*counts][:10]:
    print(count) 

## Creating the key

Alice and Bob only keep the bits where their bases match

In [0]:
alice_keys = []
bob_keys = []
eve_keys = []
matches = 0
for temp_key in [*counts]:
    temp_key_rev = temp_key[::-1] # qubit orders are reversed in qiskit
    alice_temp_key = temp_key_rev[0:num_qubits//2]
    bob_temp_key = temp_key_rev[(num_qubits//2)+1:num_qubits+1]  #Note the space 
    eve_temp_key = temp_key_rev[num_qubits+2:]
    alice_key = ''
    bob_key = ''
    eve_key = ''
    for i in range(num_qubits//2):
        if alice_basis[i] == bob_basis[i]: # Only choose bits where Alice and Bob chose the same basis
            alice_key += str(alice_temp_key[i])
            bob_key += str(bob_temp_key[i])
            eve_key += str(eve_temp_key[i])
        else:
            if alice_temp_key[i] == bob_temp_key[i]:
#                 print(alice_basis[i],bob_basis[i],alice_temp_key[i],bob_temp_key[i])
                matches += 1
    alice_keys.append(alice_key)
    bob_keys.append(bob_key)
    eve_keys.append(eve_key)

Let us measure how many of the bits where Alice and Bob chose different bases are correlated

In [0]:
def count_mismatches(a_basis,b_basis):
    return len([1 for a,b in zip(a_basis,b_basis) if a!=b])
matches/(count_mismatches(alice_basis,bob_basis)*8192)

In [0]:
# Check if their keys are the same
for alice_key, bob_key, eve_key in zip(alice_keys, bob_keys, eve_keys):
    print(alice_key,bob_key,eve_key,alice_key == bob_key)